In [5]:
import scattertext as st
import re, io
from pprint import pprint
import pandas as pd
import numpy as np
from scipy.stats import rankdata, hmean, norm
import spacy
import os, pkgutil, json, urllib
from urllib.request import urlopen
from IPython.display import IFrame
from IPython.core.display import display, HTML
from scattertext import CorpusFromPandas, produce_scattertext_explorer

In [16]:
display(HTML("<style>.container { width:98% !important; }</style>"))

In [6]:
data = pd.read_excel('python_output/data.xlsx')

In [7]:
data.head()

,Title,Tag,Time,Text
0,台灣推出迪士尼聯乘「史迪仔」彩妝！小籠包、滷肉飯趣味包裝！「湖水冰籃色」暑意全消！,Make-up,2020.06.24,迪士尼動畫《扮嘢小魔星》可以說是陪伴了不少人成長，而史迪仔（Stitch）更是人氣角色之一，...
1,台灣推出迪士尼聯乘「史迪仔」彩妝！小籠包、滷肉飯趣味包裝！「湖水冰籃色」暑意全消！,Make-up,2020.06.24,《史迪仔系列》 台灣好呷唇蜜 共2色
2,台灣推出迪士尼聯乘「史迪仔」彩妝！小籠包、滷肉飯趣味包裝！「湖水冰籃色」暑意全消！,Make-up,2020.06.24,"（單支售價：台幣 $699）/（2支組合裝售價：台幣 $1,380）"
3,台灣推出迪士尼聯乘「史迪仔」彩妝！小籠包、滷肉飯趣味包裝！「湖水冰籃色」暑意全消！,Make-up,2020.06.24,唇釉分別有#橙香蜜茶和#豆香玫瑰兩種顏色，橙調能提亮膚色，給予人活潑有好氣色的感覺，而玫瑰色...
4,【氣墊粉底推薦】韓國節目實測5款氣墊粉底！第1位遮瑕/持妝度最出色！香港有售！,Make-up,2020.06.23,近年氣墊粉底不斷升級改良，妝效可說是娉美粉底液，不管你是追求高遮瑕，還是持妝度，一款優秀的氣...


In [8]:
import spacy
nlp = spacy.load('zh_core_web_sm')

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/qt/r49rhxw54t350tf7mx0th4pc0000gn/T/jieba.cache
Loading model cost 1.728 seconds.
Prefix dict has been built successfully.


In [10]:
data['parsed'] = data.Text.apply(nlp)

In [11]:
data.head()

,Title,Tag,Time,Text,parsed
0,台灣推出迪士尼聯乘「史迪仔」彩妝！小籠包、滷肉飯趣味包裝！「湖水冰籃色」暑意全消！,Make-up,2020.06.24,迪士尼動畫《扮嘢小魔星》可以說是陪伴了不少人成長，而史迪仔（Stitch）更是人氣角色之一，...,"(迪士尼, 動畫, 《, 扮嘢, 小, 魔星, 》, 可以, 說是, 陪伴, 了, 不少, ..."
1,台灣推出迪士尼聯乘「史迪仔」彩妝！小籠包、滷肉飯趣味包裝！「湖水冰籃色」暑意全消！,Make-up,2020.06.24,《史迪仔系列》 台灣好呷唇蜜 共2色,"(《, 史迪仔, 系列, 》, 台灣, 好呷, 唇蜜, 共, 2, 色)"
2,台灣推出迪士尼聯乘「史迪仔」彩妝！小籠包、滷肉飯趣味包裝！「湖水冰籃色」暑意全消！,Make-up,2020.06.24,"（單支售價：台幣 $699）/（2支組合裝售價：台幣 $1,380）","(（, 單, 支售, 價, ：, 台幣, $, 699, ）, /, （, 2, 支, 組合..."
3,台灣推出迪士尼聯乘「史迪仔」彩妝！小籠包、滷肉飯趣味包裝！「湖水冰籃色」暑意全消！,Make-up,2020.06.24,唇釉分別有#橙香蜜茶和#豆香玫瑰兩種顏色，橙調能提亮膚色，給予人活潑有好氣色的感覺，而玫瑰色...,"(唇釉, 分別, 有, #, 橙香, 蜜茶, 和, #, 豆香, 玫瑰, 兩種, 顏色, ，..."
4,【氣墊粉底推薦】韓國節目實測5款氣墊粉底！第1位遮瑕/持妝度最出色！香港有售！,Make-up,2020.06.23,近年氣墊粉底不斷升級改良，妝效可說是娉美粉底液，不管你是追求高遮瑕，還是持妝度，一款優秀的氣...,"(近年, 氣墊, 粉底, 不, 斷, 升級, 改良, ，, 妝效, 可說, 是, 娉美粉, ..."


In [13]:
print("Document Count")
print(data.groupby('Tag')['Text'].count())

Document Count
Tag
Fragrance & Body    216
Hair                245
Make-up             597
Nails                 4
Skincare            606
Name: Text, dtype: int64


In [14]:
print("Word Count")
print(data.groupby('Tag').apply(lambda x: x.Text.apply(lambda x: len(x.split())).sum()))

Word Count
Tag
Fragrance & Body     696
Hair                 482
Make-up             1567
Nails                  4
Skincare            1445
dtype: int64


In [12]:
corpus = st.CorpusFromPandas(data,
                             category_col='Tag',
                             text_col='Text',
                             nlp=nlp).build()

In [15]:
html = produce_scattertext_explorer(corpus,
                                    category='Make-up',
                                    category_name='Democratic',
                                    not_category_name='Republican',
                                    width_in_pixels=1000,
                                    minimum_term_frequency=5,
                                    transform=st.Scalers.scale,
                                    metadata=data['Title'])
file_name = 'python_output/Test1.html'
open(file_name, 'wb').write(html.encode('utf-8'))
IFrame(src=file_name, width = 1200, height=700)

In [24]:
html = st.produce_scattertext_explorer(corpus,
                                       category='Hair',
                                       category_name='Hair',
                                       not_category_name='Other',
                                       minimum_term_frequency=5,
                                       width_in_pixels=1000,
                                       transform=st.Scalers.log_scale_standardize)
file_name = 'python_output/Test2.html'
open(file_name, 'wb').write(html.encode('utf-8'))
IFrame(src=file_name, width = 1200, height=700)

In [25]:
html = st.produce_scattertext_explorer(corpus,
                                       category='Make-up',
                                       category_name='Make up',
                                       not_category_name='Other',
                                       minimum_term_frequency=5,
                                       width_in_pixels=1000,
                                       transform=st.Scalers.log_scale_standardize)
file_name = 'python_output/Test2.html'
open(file_name, 'wb').write(html.encode('utf-8'))
IFrame(src=file_name, width = 1200, height=700)

In [21]:
html = produce_scattertext_explorer(corpus,
                                    category='Hair',
                                    category_name='Democratic',
                                    not_category_name='Republican',
                                    width_in_pixels=1000,
                                    minimum_term_frequency=5,
                                    transform=st.Scalers.percentile,
                                    metadata=data['Title'])
file_name = 'python_output/Test3.html'
open(file_name, 'wb').write(html.encode('utf-8'))
IFrame(src=file_name, width = 1200, height=700)